In [59]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [60]:
# restore all of our data structures
import pickle
tf.reset_default_graph()
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [61]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [62]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w.lower() == s.lower(): 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [63]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[u'age_neg', u'age_pos', u'facebook_neg', u'facebook_pos', u'goodbye', u'grade_neg', u'grade_pos', u'home_neg', u'home_pos', u'location_neg', u'location_pos', u'name', u'pets_neg', u'pets_pos', u'subscribe_neg', u'subscribe_pos', u'youtube_neg', u'youtube_pos']


In [64]:
# load our saved model

model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /home/salts/cybersecurity-exhibition/chatbot/model.tflearn


In [78]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.05
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    print(results)
    # if we have a classification then find the matching intent tag
    
    
    
    if (userID in context and context[userID] is not None):
        for j in results:
            for i in intents["intents"]:
                if i['tag'] == j[0] and 'context_filter' in i and i["context_filter"] == context[userID]:
                    if ('context_set' in i):
                        context[userID] = i['context_set']
                    else:
                        context[userID] = None
                    return random.choice(i["responses"])
                    
        
        for i in intents['intents']:
            if ('context_filter' in i and i["context_filter"] == context[userID]):
                if ('context_set' in i):
                    context[userID] = i['context_set']
                else:
                    context[userID] = None
                return random.choice(i["responses"])
    
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    contextValue = None
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        contextValue = i['context_set']
                        

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        if (contextValue is not None):
                            context[userID] = i['context_set']
                        return random.choice(i['responses'])
                    
                    if (contextValue is not None):
                        context[userID] = i['context_set']

            results.pop(0)

In [79]:
print(response('hei'))

[(u'name', 0.9437405)]
Tere, mis su nimi on? 😀😀😀


In [87]:
context = {'123': None}

print(response("hei"))
print(response("Ei ütle"))
print(context)
print(response("Juhan"))
print(context)
print(response("Olen"))
print(response("Jah"))
print(response("Kass"))
print(response("Peeter123"))
print(response("Peeter.Kasemets"))
print(response("11 klass"))
print(response("Nägemist!"))

[(u'name', 0.9437405)]
Tere, mis su nimi on? 😀😀😀
[(u'grade_neg', 0.24722482), (u'home_neg', 0.1944533), (u'location_neg', 0.19024575), (u'age_neg', 0.14808697), (u'subscribe_neg', 0.11336359), (u'facebook_neg', 0.06272742)]
Ära karda, ma tunnen su vanemaid, kui vana sa oled? 😀
{'123': u'age'}
[(u'subscribe_pos', 0.28498334), (u'youtube_pos', 0.17598271), (u'facebook_pos', 0.17084652), (u'age_pos', 0.13873255), (u'pets_pos', 0.10571235), (u'home_pos', 0.09153214)]
Juba nii vana! Kas sa mäletad mind? Mul on kodus koer ja me elame perega Viimsis suures majas. Kus sa elad?
{'123': u'home'}
[(u'home_pos', 0.43035284), (u'subscribe_pos', 0.26698264), (u'age_pos', 0.1213392), (u'youtube_pos', 0.071657375), (u'grade_pos', 0.05511445)]
Ära muretse, tean kus sa oled. Kas su vanemad on kodus?
[(u'pets_pos', 0.31660372), (u'subscribe_pos', 0.2744106), (u'youtube_pos', 0.19191636), (u'facebook_pos', 0.13534252)]
Okei 😂 Kas sul ka lemmikloomi on?
[(u'youtube_pos', 0.97259766)]
Väga huvitav. Tead, mu

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 1: ordinal not in range(128)

In [10]:

context

{'123': u'name'}

In [11]:
print(response('Peeter'))

[(u'age_pos', 0.48259798), (u'pets', 0.19497152), (u'name', 0.16151457)]
😀 Tore! Ma olen Martin, 13. Vana sa?


In [12]:
context

{'123': u'name'}

In [13]:
print(response('15'))

[(u'age_pos', 0.48259798), (u'pets', 0.19497152), (u'name', 0.16151457)]
Rõõm tutvuda, ma olen Mari ja olen 14-aastane. Kui vana sa oled?


In [14]:
print(response("Tallinnas"))

[(u'pets', 0.884738)]
💩 Kahju! Ma olen Martin, 13. Vana sa?


In [15]:
print(response("Pelgulinnas"))

[(u'age_pos', 0.48259798), (u'pets', 0.19497152), (u'name', 0.16151457)]
Okei! Mul on kodus koer ja me elame perega Viimsis suures majas. Kus sa elad?


In [16]:
print(response("jah, koer"))

[(u'age_pos', 0.48259798), (u'pets', 0.19497152), (u'name', 0.16151457)]
Okei! Kus su kodu muidu asub?


In [17]:
print(response("nimi123"))

[(u'age_pos', 0.48259798), (u'pets', 0.19497152), (u'name', 0.16151457)]
Olgu, kuule, kas sul ka mõni lemmikloom on?


In [18]:
print(response("nimi123"))

[(u'age_pos', 0.48259798), (u'pets', 0.19497152), (u'name', 0.16151457)]
Okei 😂 Kas sul ka lemmikloomi on?


In [19]:
print(response("3"))

[(u'age_pos', 0.48259798), (u'pets', 0.19497152), (u'name', 0.16151457)]
Okei 😂 Kas sul ka lemmikloomi on?


In [20]:
classify('today')

[(u'age_pos', 0.48259798), (u'pets', 0.19497152), (u'name', 0.16151457)]

In [21]:
# clear context
response("Hi there!", show_details=True)

[(u'goodbye', 0.6365932), (u'age_pos', 0.23290391), (u'facebook', 0.07030181)]


u'Olgu, kuule, kas sul ka m\xf5ni lemmikloom on?'

In [33]:
response('today')
classify('today')

[]

In [34]:
response("thanks, your great")

In [23]:
context = {}